In [1]:
import os
import rosbag
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from cv_bridge import CvBridge
bridge = CvBridge()
from PIL import Image
from tqdm import tqdm
import io
import pointcloud2 as pc2
from skimage.io import imsave

In [3]:
def numpy_image(data: any, dims: tuple) -> np.ndarray:
    """
    Return a NumPy tensor from image data.

    Args:
        data: the image data to convert to a NumPy tensor
        dims: the height and width of the image

    Returns:
        an RGB or RGBA NumPy tensor of the image data

    """
    # try to create an RGBA tensor from the image data
    try:
        return np.array(data, dtype='uint8').reshape((*dims, 4))
    # try to create an RGB tensor from the image data
    except ValueError:
        return np.array(data, dtype='uint8').reshape((*dims, 3))


def get_camera_image(data: bytes, dims: tuple) -> np.ndarray:
    """
    Get an image from binary ROS data.

    Args:
        data: the binary data to extract an image from
        dims: the expected dimensions of the image

    Returns:
        an uncompressed NumPy tensor with the 8-bit RGB pixel data

    """
    try:
        # open the compressed image using Pillow
        with Image.open(io.BytesIO(data)) as rgb_image:
            return numpy_image(rgb_image, dims)
    # if an OS error happens, the image is raw data
    except OSError:
        return numpy_image(list(data), dims)

In [4]:
def extract_pointcloud(msg):
    points = pc2.read_points(msg, skip_nans=True, field_names=('x', 'y', 'z', 'intensity'))
    result = np.array([[x, y, z, i] for x, y, z, i in points])
    return result

In [5]:
data_path = '/media/kirill/2E493B983C144ACC/rosbags/husky_rosbags/toposlam_bags'
output_dir = '/home/kirill/TopoSLAM/OpenPlaceRecognition/data/floor5_old'
#bags = os.listdir(data_path)
#bags.sort()
bags = ['5th_floor_with_loops.bag']
front_stamps_total = []
back_stamps_total = []
lidar_stamps_total = []
for bag_name in bags:
    front_images = []
    front_stamps = []
    back_images = []
    back_stamps = []
    lidar_clouds = []
    lidar_stamps = []
    bag = rosbag.Bag(os.path.join(data_path, bag_name))
    for topic, msg, t in tqdm(bag.read_messages()):
        if topic == '/zed_node/left/image_rect_color/compressed':
            img = get_camera_image(msg.data, (720, 1280))
            front_images.append(img)
            front_stamps.append(int(t.to_sec() * 1e9))
    if not os.path.exists(os.path.join(output_dir, 'front_images')):
        os.mkdir(os.path.join(output_dir, 'front_images'))
    for img, stamp in tqdm(zip(front_images, front_stamps)):
        pil_image = Image.fromarray(img)
        pil_image.save(os.path.join(output_dir, 'front_images', '{}.png'.format(stamp)))
    front_images = []
    bag = rosbag.Bag(os.path.join(data_path, bag_name))
    for topic, msg, t in tqdm(bag.read_messages()):
        if topic == '/realsense_back/color/image_raw/compressed':
            img = get_camera_image(msg.data, (720, 1280))
            back_images.append(img)
            back_stamps.append(int(t.to_sec() * 1e9))
    if not os.path.exists(os.path.join(output_dir, 'back_images')):
        os.mkdir(os.path.join(output_dir, 'back_images'))
    for img, stamp in tqdm(zip(back_images, back_stamps)):
        pil_image = Image.fromarray(img)
        pil_image.save(os.path.join(output_dir, 'back_images', '{}.png'.format(stamp)))
    back_images = []
    bag = rosbag.Bag(os.path.join(data_path, bag_name))
    for topic, msg, t in tqdm(bag.read_messages()):
        if topic == '/velodyne_points':
            pcd = extract_pointcloud(msg)
            lidar_clouds.append(pcd)
            lidar_stamps.append(int(t.to_sec() * 1e9))
    if not os.path.exists(os.path.join(output_dir, 'lidar_clouds')):
        os.mkdir(os.path.join(output_dir, 'lidar_clouds'))
    for cloud, stamp in tqdm(zip(lidar_clouds, lidar_stamps)):
        cloud.tofile(os.path.join(output_dir, 'lidar_clouds', '{}.bin'.format(stamp)), sep=',', format="%f")
        #cloud_fromfile = np.fromfile(os.path.join(output_dir, 'lidar_clouds', '{}.bin'.format(stamp)), sep=',', dtype=float, count=cloud.shape[0] * 4).reshape((-1, 4))
        #np.savetxt(os.path.join(output_dir, 'lidar_clouds', '{}.txt'.format(stamp)), cloud)
    front_stamps_total += front_stamps
    back_stamps_total += back_stamps
    lidar_stamps_total += lidar_stamps
np.savetxt(os.path.join(output_dir, 'front_timestamps.txt'), front_stamps_total)
np.savetxt(os.path.join(output_dir, 'back_timestamps.txt'), back_stamps_total)
np.savetxt(os.path.join(output_dir, 'lidar_timestamps.txt'), lidar_stamps_total)

124816it [02:51, 727.17it/s]
3883it [10:14,  6.32it/s]
124816it [03:12, 646.99it/s]
5867it [18:25,  5.31it/s]
124816it [06:58, 297.96it/s]
3880it [02:18, 27.98it/s]
